In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pypfopt import expected_returns, risk_models, BlackLittermanModel
import yfinance as yf
import pypfopt
import os

In [76]:
data = {}
start_date = '2020-01-01'
end_date = '2020-12-31'

In [77]:
etfs = pd.read_csv('data/etfs.csv')

In [78]:
for i in range(len(etfs)):
    data[etfs.iloc[i, 0]] = yf.download(etfs.iloc[i, 0], period='max')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [79]:
volatility_etfs = {}
for ticker in data:
    volatility_etfs[ticker] =1- (np.log(data[ticker]['Close']/data[ticker]['Close'].shift(-1))).std()*np.sqrt(252)
#    volatility_etfs[ticker] = 1- np.log(data[ticker]['Close'].pct_change()).dropna()

In [80]:
volatility_etfs

{'BLV': 0.8867535563819869,
 'VUSTX': 0.891197280752484,
 'VNQ': 0.6968778735290573,
 'VPU': 0.8152927698536916,
 'VTI': 0.8042887580540653,
 'JUST': 0.7824342571251431,
 'BSV': 0.9689143790955306,
 'VGSH': 0.9882393857073164,
 'VO': 0.7874011035100767,
 'IVOG': 0.8034412359913998,
 'VBK': 0.7654636990475332,
 'VIOG': 0.7850324773431735}

In [81]:
for ticker in data:
    data[ticker].to_csv('data/' + ticker + '.csv')

for ticker in data:
    data[ticker]['percentageChange'] = data[ticker]['Close'].pct_change()*10
    data[ticker]['percentageChange'] = data[ticker]['percentageChange'].fillna(0)

In [82]:
onlyClosePrices = pd.DataFrame()
for ticker in data:
    onlyClosePrices[ticker] = data[ticker]['Close']

In [83]:
S = pypfopt.risk_models.CovarianceShrinkage(onlyClosePrices).ledoit_wolf()

In [84]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import read_csv
import math
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.layers.core import Dense, Activation, Dropout
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
import time #helper libraries

In [85]:
scalar = MinMaxScaler(feature_range=(-1, 1))

In [86]:
for ticker in data:
    data[ticker] = scalar.fit_transform(data[ticker])

In [87]:
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), -2]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [88]:
trainingData = {}
testData = {}

In [89]:
for ticker in data:
    trainingData[ticker], testData[ticker] = train_test_split(data[ticker], test_size=0.05, shuffle=False)

In [90]:
trainX, trainY = {}, {}
testX, testY = {}, {}
timeStep = 20
for ticker in data:
    trainX[ticker], trainY[ticker] = create_dataset(trainingData[ticker], timeStep)
    testX[ticker], testY[ticker] = create_dataset(testData[ticker], timeStep)

In [91]:
trainX['VTI'].shape

(5000, 20)

In [92]:
models = {}
for ticker in data:
    models[ticker] = Sequential()
    models[ticker].add(LSTM(units=100, return_sequences=True, input_shape=(trainX['VTI'].shape[1], 1)))
    models[ticker].add(Dropout(0.2))
    models[ticker].add(LSTM(50, return_sequences=False))
    models[ticker].add(Dropout(0.2))
    models[ticker].add(Dense(1))
    models[ticker].add(Activation('linear'))
    models[ticker].compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.8), metrics=[tf.keras.metrics.RootMeanSquaredError()])
"""model = Sequential()
model.add(LSTM(units=100, return_sequences=True, input_shape=(trainX['VTI'].shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.8), metrics=[tf.keras.metrics.RootMeanSquaredError()])
#model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9), metrics=[tf.keras.metrics.RootMeanSquaredError()])"""

"model = Sequential()\nmodel.add(LSTM(units=100, return_sequences=True, input_shape=(trainX['VTI'].shape[1], 1)))\nmodel.add(Dropout(0.2))\nmodel.add(LSTM(units=50, return_sequences=True))\nmodel.add(Dropout(0.2))\nmodel.add(LSTM(units=50, return_sequences=True))\nmodel.add(Dropout(0.2))\nmodel.add(LSTM(units=50))\nmodel.add(Dropout(0.2))\nmodel.add(Dense(units=1))\nmodel.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.8), metrics=[tf.keras.metrics.RootMeanSquaredError()])\n#model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9), metrics=[tf.keras.metrics.RootMeanSquaredError()])"

In [93]:
VERSION = '4'
for ticker in data:
    models[ticker].fit(trainX[ticker], trainY[ticker], epochs=50*2, batch_size=5, verbose=1, validation_data=(testX[ticker], testY[ticker]))
    models[ticker].save('models/' + ticker + '_' + VERSION + '.h5')
    models[ticker].evaluate(testX[ticker], testY[ticker])
    models[ticker].save_weights('weights/' + ticker + '_' + VERSION + '_weights.h5')

Epoch 1/100
723/723 [==============================] - 9s 10ms/step - loss: 0.0294 - root_mean_squared_error: 0.1713 - val_loss: 0.0911 - val_root_mean_squared_error: 0.3018
Epoch 2/100
723/723 [==============================] - 7s 9ms/step - loss: 0.0256 - root_mean_squared_error: 0.1600 - val_loss: 0.1109 - val_root_mean_squared_error: 0.3330
Epoch 3/100
723/723 [==============================] - 7s 9ms/step - loss: 0.0240 - root_mean_squared_error: 0.1550 - val_loss: 0.0742 - val_root_mean_squared_error: 0.2723
Epoch 4/100
723/723 [==============================] - 7s 9ms/step - loss: 0.0223 - root_mean_squared_error: 0.1495 - val_loss: 0.1318 - val_root_mean_squared_error: 0.3630
Epoch 5/100
723/723 [==============================] - 7s 9ms/step - loss: 0.0220 - root_mean_squared_error: 0.1482 - val_loss: 0.0571 - val_root_mean_squared_error: 0.2389
Epoch 6/100
723/723 [==============================] - 7s 9ms/step - loss: 0.0215 - root_mean_squared_error: 0.1465 - val_loss: 0.1729

In [ ]:
model = load_model('VTI.h5')

In [22]:
viewDict = {}

In [24]:
models = {}
for ticker in data:
    print(ticker)
    models[ticker] = load_model(ticker + '.h5')
    viewDict[ticker] = model.predict(testX[ticker])

VTI
VTV
VOE
VBR
GBIL


ValueError: Unexpected result of `predict_function` (Empty batch_outputs). Please use `Model.compile(..., run_eagerly=True)`, or `tf.config.run_functions_eagerly(True)` for more information of where went wrong, or file a issue/bug to `tf.keras`.

In [ ]:
trainX['VTI'].shape

In [ ]:
viewDict['VTI']

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pypfopt
import yfinance as yf
import pandas_datareader as pdr
import seaborn as sns
import random

In [24]:
netAssets = {}
for ticker in data:
    netAssets[ticker] = yf.Ticker(ticker).info['totalAssets']

In [25]:
netAssets

{'BLV': 8635350016,
 'VUSTX': 3014977536,
 'VNQ': 78395367424,
 'VPU': 7854255104,
 'VTI': 1207961518080,
 'JUST': 267417440,
 'BSV': 67210063872,
 'VGSH': 17471244288,
 'VO': 145426268160,
 'IVOG': 878269056,
 'VBK': 28689035264,
 'VIOG': 488499392}

In [29]:
csvAssets = pd.DataFrame.from_dict(netAssets,orient='index')
csvAssets.to_csv('data/assets.csv')

In [ ]:
csvAssets = pd.read_csv('assets.csv', index_col=0)

In [ ]:
csvAssets['0']['VTI']

In [ ]:
delta = pypfopt.black_litterman.market_implied_risk_aversion(pd.Series(netAssets), risk_free_rate=0.05796)

In [ ]:
marketPrior = pypfopt.black_litterman.market_implied_prior_returns(netAssets, delta, S)

In [ ]:
viewDict = {}
confidence = []
for ticker in data:
    pred = models[ticker].predict(testX[ticker])
    viewDict[ticker] = (pred[-1,0] - testX[ticker][-2, 0])/testX[ticker][-2, 0]
    confidence.append(1-model.evaluate(testX[ticker], testY[ticker])[1])

In [ ]:
viewDict

In [ ]:
bl = pypfopt.black_litterman.BlackLittermanModel(S, pi=marketPrior, absolute_views=viewDict, rho=delta)

In [ ]:
bl_confi = pypfopt.BlackLittermanModel(S, absolute_views=viewDict, omega="idzorek", rho=delta, view_confidences=confidence)

In [ ]:
bl_return_confi = bl_confi.bl_returns()
bl_return_confi.name = 'BL Returns with Confidence'

In [ ]:
S_bl_confi = bl_confi.bl_cov()

In [ ]:
ef = pypfopt.EfficientFrontier(bl_return_confi, S_bl_confi, weight_bounds=(0, 0.2))

In [ ]:
#ef.add_objective('Sharpe', 'max')
weights = ef.min_volatility()
ef.portfolio_performance(verbose=True)

In [ ]:
import json
json.dumps(weights)

In [ ]:
wt_min_vola = pd.DataFrame([weights], columns=weights.keys()).T*100
print ('Weights in Percentage ********************')
print(wt_min_vola.round(4))

In [ ]:
data